# 검증 세트

검증 세트 : 테스트 세트를 사용하지 않고 훈련 세트를 나누는 방법

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
# class 열을 타깃으로 사용하고, 나머지 열은 특성에 배정
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련 세트의 입력 데이터와 타깃 데이터를 trian_input 과 train_target 배열에 저장
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
# test_size 매개변수를 0.2로 지정
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증

교차 검증 : 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터 사용 가능

사이킷런 : cross_validate() 교차 검증 함수 사용. 훈련 세트 전체를 cross_validate() 함수에 전달

In [6]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
# cross_validate 는 5-폴드 교차 검증 수행
# cv 매개변수에서 폴드 수를 바꿀 수도 있음
print(scores)

{'fit_time': array([0.02269912, 0.02428222, 0.02751279, 0.0416193 , 0.02219915]), 'score_time': array([0.00850749, 0.00480604, 0.00218296, 0.00203896, 0.00179791]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np

print(np.mean(scores['test_score']))
# 교차 검증의 최종 점수 : test_score 키에 담긴 5개 점수의 평균

0.855300214703487


In [8]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


주의 : cross_validate() 는 훈련 세트를 섞어 폴드를 나누지 않음.

교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야 함.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 지정함.

cross_validate() 함수는 기본적으로 회귀 모델에서 KFold 분할기 사용

분류 모델의 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 를 사용

In [9]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행
# n_splits 매개변수는 몇 폴드 교차 검증을 할지 결정한다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝

머신러닝 모델이 학습하는 파라미터 : 모델 파라미터

모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 : 하이퍼파라미터

사이킷런에서 제공하는 '그리드 서치' 사용

사이킷런 GridSearchCV 클래스 : 하이퍼파라미터 탐색과 교차 검증 한 번에 수행 , cross_validate() 함수 호출 필요 없음.

In [10]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [11]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

GridSearchCV 의 cv 매개변수 기본 값 5

min_impurity_decrease 값마다 5-폴드 교차 검증 수행 : 5 x 5 = 25개의 모델을 훈련.


In [12]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

best_estimator_ 속성
검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 모델을 다시 훈련

In [13]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [14]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [15]:
print(gs.cv_results_['mean_test_score'])
# 첫번째 값이 가장 큰 것을 알 수 있음.

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [16]:
# argmax 함수 사용하여 가장 큰 값의 인덱스 추출
# 인덱스 사용하여 params 키에 저장된 매개변수 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치 수행하여 최상의 평균 점수가 나오는 매개변수 조합 찾음. 이 조합은 그리드 서치 객채에 저장됨.
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련.

In [17]:
# max_depth 로 트리의 깊이 제한, min_samples_split 으로 노드를 나누기 위한 최소 샘플수 결정
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [19]:
print(gs.best_params_) # 최상의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [20]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수

0.8683865773302731


# 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 정하기 어려워 그리드 서치 수행 시간이 오래걸릴 때, 랜덤 서치 사용.

랜덤 서치 : 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체 전달.

In [21]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([8, 3, 8, 9, 7, 5, 4, 8, 5, 6])

In [22]:
np.unique(rgen.rvs(1000), return_counts=True) # 난수 발생

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101,  98,  90,  91,  98, 126, 119, 101,  93,  83]))

In [23]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.91750274, 0.70793263, 0.73874442, 0.82821701, 0.26089594,
       0.29948732, 0.14282905, 0.50952598, 0.88948522, 0.43734766])

In [25]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [24]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(5, 20),
                                        'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                                        'min_samples_split': range(2, 100, 10)},
                   random_state=42)

In [26]:
print(gs.best_params_) # 최적의 매개변수 조합

{'min_samples_split': 12, 'min_impurity_decrease': 0.0005, 'max_depth': 11}


In [27]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최적의 교차 검증 점수

0.8681935292811135


In [28]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.8615384615384616
